# Finding suspicious behavior by tracking down outliers

To reproduce this finding from the Dallas Morning News, we'll need to use standard deviation, regression, and residuals to identify schools that performed suspiciously well in certain standardized tests.

### Prep work: Downloading necessary files

Before we get started, let's talk about the data we'll be using. **TAKS** is a standardized test that was given to all students in Texas public schools

* **cfy04e4.dat:** 2004 fourth-grade TAKS scores - standardized test scores for 2004's fourth-graders
* **cfy03e3.dat:** 2003 third-grade TAKS scores - standardized test scores for 2003's third-graders
* **cfy04e5.dat:** 2004 fifth-grade TAKS scores - standardized test scores for 2004's fifth-graders
* **cfy04e3.dat:** 2004 third-grade TAKS scores - standardized test scores for 2004's third-graders


In [48]:
!pip install seaborn
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

pd.set_option("display.max_rows", 200)
pd.set_option("display.max_columns", 200)

You should consider upgrading via the 'pip install --upgrade pip' command.


# Reading in our data

We'll start by opening up our dataset - standardized test performance at each school, for fourth graders in 2004 and third graders in 2003.

In [49]:
fourth_graders = pd.read_csv("cfy04e4.dat", usecols=['r_all_rs', 'CNAME', 'CAMPUS'])
fourth_graders = fourth_graders.add_suffix('_fourth')
fourth_graders.head(3)

,CAMPUS_fourth,CNAME_fourth,r_all_rs_fourth
0,1902103,CAYUGA EL,2392.0
1,1903101,ELKHART EL,2263.0
2,1904102,FRANKSTON EL,2242.0


Read in `cfy03e3.dat` the same way we read in `cfy04e4` above. **You only want the `CAMPUS` and `r_all_rs` columns.**

* `03` is the year
* `e3` means third grade
* `r_all_rs` means reading scores

In [50]:
third_graders = pd.read_csv("cfy03e3.dat", usecols=['r_all_rs', 'CAMPUS'])
third_graders = third_graders.add_suffix('_third')
third_graders.head(3)

,CAMPUS_third,r_all_rs_third
0,1902103,2330.0
1,1903101,2285.0
2,1904102,2299.0


Merge these two dataframes to create a dataframe called `merged`. It should have at least three columns:

* `CNAME_fourth`, the school name
* `r_all_rs_fourth`, the reading scores from 2004's fourth graders
* `r_all_rs_third`, the reading scores from 2003's third graders

You'll probably have more than that, so you're welcome to drop the other columns if you'd like!

To double-check: your first row should be `CAYUGA EL` with a fourth-grader score of 2392 and a third-grade score of 2330.

In [51]:
merged = fourth_graders.merge(third_graders, left_on = 'CAMPUS_fourth', right_on = 'CAMPUS_third')
merged.head()

,CAMPUS_fourth,CNAME_fourth,r_all_rs_fourth,CAMPUS_third,r_all_rs_third
0,1902103,CAYUGA EL,2392.0,1902103,2330.0
1,1903101,ELKHART EL,2263.0,1903101,2285.0
2,1904102,FRANKSTON EL,2242.0,1904102,2299.0
3,1906102,NECHES EL,2218.0,1906102,2236.0
4,1907110,STORY EL,2200.0,1907110,2202.0


# Using a regression to predict fourth-grade scores

The Dallas Morning News decided to run a **regression**, which is a way of predicting how two different variables interact. In this case, we want to see the relationship between a **third grade score** and a **fourth grade score**. 

We will assume that if you did poorly as a third-grader in 2003 you'd probably do poorly as a fourth-grader in 2004, and if you did well as a third-grader in 2003 you'd probably do well as a fourth-grader in 2004.

Use statsmodels to **find the relationship is between third-grade scores and fourth-grade scores.**

*Tip: Before you start, you'll want to drop all of your rows that are missing data. `merged = merged.dropna()` should take care of it for you, but you'll probably also want to see how many rows you're losing.*

In [52]:
merged = merged.dropna()

In [53]:
import statsmodels.formula.api as smf

model = smf.ols('r_all_rs_fourth ~ r_all_rs_third', data=merged)
results = model.fit()

results.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:        r_all_rs_fourth   R-squared:                       0.647
Model:                            OLS   Adj. R-squared:                  0.647
Method:                 Least Squares   F-statistic:                     6410.
Date:                Fri, 19 Feb 2021   Prob (F-statistic):               0.00
Time:                        09:52:13   Log-Likelihood:                -17961.
No. Observations:                3501   AIC:                         3.593e+04
Df Residuals:                    3499   BIC:                         3.594e+04
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
==================================================================================
                     coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------
Intercept        632.6929     19.936     31.736      0.000     593.606     671.780
r_all_rs_third     0.7094      0.009     80.061      0.000       0.692       0.727
==============================================================================
Omnibus:                      263.054   Durbin-Watson:                   1.897
Prob(Omnibus):                  0.000   Jarque-Bera (JB):             1312.900
Skew:                           0.147   Prob(JB):                    8.08e-286
Kurtosis:                       5.986   Cond. No.                     6.49e+04
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 6.49e+04. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

In [54]:
# Based on third grade scores, what are the fourth grade scores
# R-squared --> 65% of variance can be explained by the regression

What does the summary above tell us? **Can we trust the results?**

In [55]:
# Yes! r-squared is good, p-value is good!

In [56]:
results.predict()

array([2285.61612572, 2253.69271496, 2263.62444276, ..., 2130.2555267 ,
       2109.68266199, 2196.2305756 ])

We aren't that interested in the relationship, really. What we're interested in is: **if we know what third-grade score a school got, we can try to predict its fourth-grade score.**

After you run a regression, you can use `results.predict()` to return the *predicted value* for each row. Not what the actual value fourth grade score was, but what the regression thinks it *should be* based on the third grade score.

**Save the prediction in a new column called `predicted_fourth`.**

In [57]:
merged['predicted_fourth'] = results.predict()
merged.head()

,CAMPUS_fourth,CNAME_fourth,r_all_rs_fourth,CAMPUS_third,r_all_rs_third,predicted_fourth
0,1902103,CAYUGA EL,2392.0,1902103,2330.0,2285.616126
1,1903101,ELKHART EL,2263.0,1903101,2285.0,2253.692715
2,1904102,FRANKSTON EL,2242.0,1904102,2299.0,2263.624443
3,1906102,NECHES EL,2218.0,1906102,2236.0,2218.931668
4,1907110,STORY EL,2200.0,1907110,2202.0,2194.811757


The predicted reading test score for Cayuga Elementary should be something like 2285.

## Using standard deviations with regression error

Notice how there's a difference between the _actual_ fourth-grade score and the _predicted_ fourth-grade score. This is called the **error** or **residual**. The bigger the error, the bigger the difference between what was expected and what actually happened.

In the actual article, they were were suspicious of one school because it performed normally, but then performed *really well*. A school like that is going to have a really big error!

If you want to look at the residual of each prediction, you can either subtract the actual from the predicted or just look at `results.resid`.

Look at `results.resid`.

In [60]:
merged['perform_diff'] = results.resid
# difference between predicted value and actual value

Some schools overperformed the prediction, some schools underperformed, but honestly those numbers don't mean anything to me! Is 106 a really big improvement from third grade to fourth grade? No idea!

To calculate what a *suspiciously* large error is, we're going to use our old friend **standard deviation.** In this case, we're going to use standard deviation to see how *far each school's error is from the average error.*

Here's the code for this one:

```py
results.resid / np.sqrt(results.mse_resid)
```

Store it in a new column called `error_std_dev`.

In [61]:
results.resid / np.sqrt(results.mse_resid)
merged['error_std_dev'] = results.resid / np.sqrt(results.mse_resid)
merged.head()

,CAMPUS_fourth,CNAME_fourth,r_all_rs_fourth,CAMPUS_third,r_all_rs_third,predicted_fourth,performance_diff,perform_diff,error_std_dev
0,1902103,CAYUGA EL,2392.0,1902103,2330.0,2285.616126,2.600187,106.383874,2.600187
1,1903101,ELKHART EL,2263.0,1903101,2285.0,2253.692715,0.227484,9.307285,0.227484
2,1904102,FRANKSTON EL,2242.0,1904102,2299.0,2263.624443,-0.528535,-21.624443,-0.528535
3,1906102,NECHES EL,2218.0,1906102,2236.0,2218.931668,-0.022771,-0.931668,-0.022771
4,1907110,STORY EL,2200.0,1907110,2202.0,2194.811757,0.126809,5.188243,0.126809


The more standard deviations away from the mean a school's error is, the bigger the gap between actual and predicted scores, and **the more suspicious its fourth-grade performance is.**

Let's look at the **top ten schools** in terms of standard deviation.

In [62]:
merged.sort_values(by = 'error_std_dev', ascending = False).head(10)


,CAMPUS_fourth,CNAME_fourth,r_all_rs_fourth,CAMPUS_third,r_all_rs_third,predicted_fourth,performance_diff,perform_diff,error_std_dev
746,57905115,HARRELL BUDD EL,2470.0,57905115,2140.0,2150.828391,7.801048,319.171609,7.801048
2261,123803101,TEKOA ACADEMY O,2313.0,123803101,2021.0,2066.408705,6.027073,246.591295,6.027073
96,15803101,HIGGS/CARTER/KI,2349.0,15803101,2097.0,2120.323799,5.589200,228.676201,5.589200
1737,101912172,HENDERSON N EL,2324.0,101912172,2093.0,2117.486162,5.047518,206.513838,5.047518
2708,180901101,MIMI FARLEY ELE,2448.0,180901101,2294.0,2260.077397,4.593119,187.922603,4.593119
697,57825001,PINNACLE SCHOOL,2274.0,57825001,2068.0,2099.750934,4.258917,174.249066,4.258917
2348,131001103,SARITA EL,2509.0,131001103,2424.0,2352.300584,3.829976,156.699416,3.829976
1538,101823101,CHILDREN FIRST,2252.0,101823101,2066.0,2098.332116,3.755881,153.667884,3.755881
1110,61911111,PRAIRIE VIEW EL,2389.0,61911111,2261.0,2236.666896,3.723257,152.333104,3.723257
801,57905185,WILLIAM B MILLE,2295.0,57905185,2129.0,2143.024891,3.714507,151.975109,3.714507


In [63]:
merged[merged['error_std_dev'] >= 3.5]

,CAMPUS_fourth,CNAME_fourth,r_all_rs_fourth,CAMPUS_third,r_all_rs_third,predicted_fourth,performance_diff,perform_diff,error_std_dev
96,15803101,HIGGS/CARTER/KI,2349.0,15803101,2097.0,2120.323799,5.589200,228.676201,5.589200
342,19901102,DEKALB EL,2356.0,19901102,2223.0,2209.709349,3.575570,146.290651,3.575570
697,57825001,PINNACLE SCHOOL,2274.0,57825001,2068.0,2099.750934,4.258917,174.249066,4.258917
746,57905115,HARRELL BUDD EL,2470.0,57905115,2140.0,2150.828391,7.801048,319.171609,7.801048
801,57905185,WILLIAM B MILLE,2295.0,57905185,2129.0,2143.024891,3.714507,151.975109,3.714507
1110,61911111,PRAIRIE VIEW EL,2389.0,61911111,2261.0,2236.666896,3.723257,152.333104,3.723257
1538,101823101,CHILDREN FIRST,2252.0,101823101,2066.0,2098.332116,3.755881,153.667884,3.755881
1642,101909104,LAKEWOOD ELEMEN,2275.0,101909104,2104.0,2125.289663,3.659152,149.710337,3.659152
1737,101912172,HENDERSON N EL,2324.0,101912172,2093.0,2117.486162,5.047518,206.513838,5.047518
2261,123803101,TEKOA ACADEMY O,2313.0,123803101,2021.0,2066.408705,6.027073,246.591295,6.027073


But what's that *really mean?* Let's keep going.

# Reproducing the story

From The Dallas Morning News:

> "In statistician's lingo, these schools had at least one average score that was more than three standard deviations away from what would be predicted based on their scores in other grades or on other tests

While we've been talking about schools with a **major increase** between the two years, we're also interested in schools with a **major drop**. That could indicate cheating in 2003 and a return to "real" testing in 2004.

Let's get a list of all of our suspicious schools according to the **three standard deviations test** they performed.

* *Tip: Absolute value might come in handy here*

In [32]:
merged.sort_values(by = 'error_std_dev', ascending = False).tail(10)

,CAMPUS_fourth,CNAME_fourth,r_all_rs_fourth,CAMPUS_third,r_all_rs_third,predicted_fourth,error_std_dev
694,57817101,FOCUS LEARNING,1973.0,57817101,2079.0,2107.554435,-3.288719
2155,108912110,JOSE DE ESCANDO,2166.0,108912110,2360.0,2306.898400,-3.443775
1451,90905101,GRANDVIEW-HOPKI,2149.0,90905101,2346.0,2296.966672,-3.616535
1127,66005101,RAMIREZ EL,2033.0,66005101,2195.0,2189.845893,-3.833556
1702,101912134,CRAWFORD EL,2056.0,101912134,2240.0,2221.769304,-4.051658
1543,101840101,TWO DIMENSIONS,2076.0,101840101,2275.0,2246.598624,-4.169694
104,15819001,SHEKINAH RADIAN,1976.0,15819001,2149.0,2157.213074,-4.429128
1708,101912140,DOGAN EL,1972.0,101912140,2150.0,2157.922483,-4.544233
2370,139908101,ROXTON EL,2130.0,139908101,2388.0,2326.761855,-4.809164
2024,105802041,SAN MARCOS PREP,2025.0,105802041,2245.0,2225.316350,-4.896042


Why three standard deviations? It's the typical measure for a statistical outlier. 99.7% of values should be inside of 3 standard deviations, one being outside of that is like [a daily event happening only once a year](https://en.wikipedia.org/wiki/68%E2%80%9395%E2%80%9399.7_rule#Table_of_numerical_values).

But then they level things up a bit:

> Using a stricter standard - four standard deviations from predictions - 41 schools have suspect scores

**Let's do the same thing.**

In [64]:
merged[merged['error_std_dev'] <= -3.5]

,CAMPUS_fourth,CNAME_fourth,r_all_rs_fourth,CAMPUS_third,r_all_rs_third,predicted_fourth,performance_diff,perform_diff,error_std_dev
104,15819001,SHEKINAH RADIAN,1976.0,15819001,2149.0,2157.213074,-4.429128,-181.213074,-4.429128
1127,66005101,RAMIREZ EL,2033.0,66005101,2195.0,2189.845893,-3.833556,-156.845893,-3.833556
1451,90905101,GRANDVIEW-HOPKI,2149.0,90905101,2346.0,2296.966672,-3.616535,-147.966672,-3.616535
1543,101840101,TWO DIMENSIONS,2076.0,101840101,2275.0,2246.598624,-4.169694,-170.598624,-4.169694
1702,101912134,CRAWFORD EL,2056.0,101912134,2240.0,2221.769304,-4.051658,-165.769304,-4.051658
1708,101912140,DOGAN EL,1972.0,101912140,2150.0,2157.922483,-4.544233,-185.922483,-4.544233
2024,105802041,SAN MARCOS PREP,2025.0,105802041,2245.0,2225.316350,-4.896042,-200.316350,-4.896042
2370,139908101,ROXTON EL,2130.0,139908101,2388.0,2326.761855,-4.809164,-196.761855,-4.809164


Four standard deviations is a daily event happening [once every 43 years](https://en.wikipedia.org/wiki/68%E2%80%9395%E2%80%9399.7_rule#Table_of_numerical_values). Is it likely that twelve of these schools just had a once-in-a-lifetime change? I mean, *technically* yes.

# Finding other suspicious scores

We might assume a school that does well in reading probably also does well in math.

**What if they did well in one, but not the other?** While the school might just have a strong department in one particular field, such discripancies could be worth investigating.

Let's look at fifth graders' math and reading scores from 2004.

**Read in `cfy04e5.dat`, keeping `CNAME`, `m_all_rs`, and `r_all_rs`**.

* `m_` is for math scores
* `r_` is for reading scores.

In [29]:
scores = pd.read_csv("cfy04e5.dat", usecols=['r_all_rs', 'm_all_rs', 'CNAME'])
scores.head(3)

,CNAME,r_all_rs,m_all_rs
0,CAYUGA EL,2308.0,2317.0
1,ELKHART EL,2193.0,2153.0
2,FRANKSTON EL,2288.0,2256.0


# Running the regression

We can't be exactly sure of the relationship between math and reading scores, so we'll run a regression to figure out how the two scores typically interact.

In [36]:
# First: drop any missing values
scores = scores.dropna()

In [44]:
# Run the regression, predicting math scores from reading scores
import statsmodels.formula.api as smf

model = smf.ols('m_all_rs ~ r_all_rs', data=scores)
results = model.fit()

results.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:               m_all_rs   R-squared:                       0.735
Model:                            OLS   Adj. R-squared:                  0.735
Method:                 Least Squares   F-statistic:                     9570.
Date:                Thu, 18 Feb 2021   Prob (F-statistic):               0.00
Time:                        15:12:27   Log-Likelihood:                -18443.
No. Observations:                3452   AIC:                         3.689e+04
Df Residuals:                    3450   BIC:                         3.690e+04
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept    136.3779     21.321      6.397      0.000      94.576     178.180
r_all_rs       0.9462      0.010     97.826      0.000       0.927       0.965
==============================================================================
Omnibus:                      210.369   Durbin-Watson:                   1.687
Prob(Omnibus):                  0.000   Jarque-Bera (JB):              858.872
Skew:                           0.127   Prob(JB):                    3.15e-187
Kurtosis:                       5.430   Cond. No.                     5.46e+04
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 5.46e+04. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

And now, just like last time, we calculate how many standard deviations away the actual score was from the predicted score. Large number of standard deviations away means a school is worth a look!

In [45]:
# I'll leave this here for you!
# Your job is to filter for schools more than 3 standard deviations from the norm.
scores['error_std_dev'] = results.resid / np.sqrt(results.mse_resid)
scores.head()

,CNAME,r_all_rs,m_all_rs,error_std_dev
0,CAYUGA EL,2308.0,2317.0,-0.064167
1,ELKHART EL,2193.0,2153.0,-1.154663
2,FRANKSTON EL,2288.0,2256.0,-0.895612
3,NECHES EL,2298.0,2312.0,0.024009
4,STORY EL,2218.0,2269.0,0.670133


In [46]:
scores[scores['error_std_dev'] >= 4]

,CNAME,r_all_rs,m_all_rs,error_std_dev
1699,BOWIE EL,2265.0,2498.0,4.316536
1810,SANDERSON EL,2235.0,2696.0,8.790097
2206,MALONE EL,2104.0,2355.0,4.501120


In [47]:
scores[scores['error_std_dev'] <= -4]

,CNAME,r_all_rs,m_all_rs,error_std_dev
2035,SAN MARCOS PREP,2203.0,1968.0,-4.997376


Sanderson Elementary looks like they either have a **really** exceptional math program or something suspicious is going on.

# Review

First, we learned about using **standard deviation** as a measurement of how unusual a measurement in a data point might be. Data points that fall many standard deviations from the mean - either above or below - might be worth investigating as bad data or from other suspicious angles (cheating schools, in this case).

Then we learned how a **linear regression** can determine the relationship between two numbers. In this case, it was how third-grade scores relate to fourth-grade scores, and then how math and reading scores relate to one another. By using a regression, you can use one variable to predict what the other should be.

Finally, we used the **residual** or **error** from the regression to see how far off each prediction was. Just like we did with the original scores, we used standard deviation to find usually suspiciously large errors. Even though yes, our regression might not be perfect, times when it's _very_ wrong probably call for an investigation.

# Discussion points

Things we might talk about in class, or that I might include on the quiz are below. You don't have to answer them here.

* Why would this analysis be based on standard deviations away from the predicted value instead of just the predicted value?
* Standard deviation is how far away from the "average" a school is. Let's say you scored 3 standard deviations away from the average, but it was only a 5-point difference. What kind of situation could lead to that? Is it as important as being 3 standard deviations away but with a 50-point difference? 
* The Dallas Morning News specifically called out schools with scores "more than three standard deviations away from what would be predicted based on their scores in other grades or on other tests." Do you think they ignored schools that were 2.99 standard deviations away?
* Did _we_ ignore those schools? If we did, how could we be more cautious in the future?
* What are the pros and cons of selecting a cutoff like three standard deviations away from the predicted value? Note that [three standard deviations is a typical number in stats](https://stats.stackexchange.com/questions/311174/what-does-it-mean-when-three-standard-deviations-away-from-the-mean-i-land-ou)
* What's the difference between a school with predicted scores -3 standard deviations away as compared to +3 standard deviations away? Do we need to pay attention to both, or only one?
* What next steps should we take after we've calculated these findings?
* If a school did have a strong math department and a weak english department, they would definitely be predicted incorrectly. What happens to that school after being flagged by research like this?